In [72]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import MistralConfig, OllamaConfig
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.retrievers import AutoRetriever
from camel.embeddings import MistralEmbedding
from camel.types import StorageType
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage


In [ ]:
import os
from getpass import getpass



In [ ]:
# Set Neo4j instance
n4j = Neo4jGraph(
    url = "",
    username = "neo4j",
    password = ""
)

In [84]:
# Set up model
model = ModelFactory.create(
    model_platform=ModelPlatformType.OLLAMA,
    model_type="nuextract",
    model_config_dict=OllamaConfig(temperature=0.2, max_tokens=2048).as_dict(),
)

Ollama server started on http://localhost:11434/v1 for nuextract model.


In [74]:
# Set instance
uio = UnstructuredIO()
kg_agent = KnowledgeGraphAgent(model=model)

2025-04-27 09:32:31,858 - root - WARNING - Invalid or missing `max_tokens` in `model_config_dict`. Defaulting to 999_999_999 tokens.


In [75]:
# Set example text input
input_file_path = r"E:\Git_clone\RAG\qa_dataset\data_clean\textbooks\en\Anatomy_Gray.txt"
with open(input_file_path, 'r', encoding='utf-8') as file:
    text_example = file.read()

In [76]:
len(text_example)

2281157

In [77]:
text_example[:1000]

'What is anatomy?\n\nAnatomy includes those structures that can be seen grossly (without the aid of magnification) and microscopically (with the aid of magnification). Typically, when used by itself, the term anatomy tends to mean gross or macroscopic anatomy—that is, the study of structures that can be seen without using a microscopic. Microscopic anatomy, also called histology, is the study of cells and tissues using a microscope.\n\nAnatomy forms the basis for the practice of medicine. Anatomy leads the physician toward an understanding of a patient’s disease, whether he or she is carrying out a physical examination or using the most advanced imaging techniques. Anatomy is also important for dentists, chiropractors, physical therapists, and all others involved in any aspect of patient treatment that begins with an analysis of clinical signs. The ability to interpret a clinical observation correctly is therefore the endpoint of a sound anatomical understanding.\n\nObservation and vis

In [80]:
# Create an element from given text
element_example = uio.create_element_from_text(
    text=text_example[:1000], element_id="0"
)

In [81]:
# Let Knowledge Graph Agent extract node and relationship information
ans_element = kg_agent.run(element_example, parse_graph_elements=False)
print(ans_element)

 Nodes:

Node(id='anatomy', type='Concept')
Node(id='structures', type='Concept')
Node(id='grossly', type='Concept')
Node(id='microscopically', type='Concept')
Node(id='cells', type='Concept')
Node(id='tissues', type='Concept')
Node(id='patient\u2019s disease', type='Concept')
Node(id='clinical signs', type='Concept')

Relationships:

Relationship(subj=Node(id='anatomy', type='Concept'), obj=Node(id='structures', type='Concept'), type='includes')
Relationship(subj=Node(id='grossly', type='Concept'), obj=Node(id='structures', type='Concept'), type='can be seen without the aid of magnification')
Relationship(subj=Node(id='microscopic anatomy', type='Concept'), obj=Node(id='cells', type='Concept'), type='is the study of cells and tissues using a microscope')
Relationship(subj=Node(id='anatomy', type='Concept'), obj=Node(id='patient\u2019s disease', type='Concept'), type='leads the physician towards an understanding of')
Relationship(subj=Node(id='anatomy', type='Concept'), obj=Node(id='cl

In [ ]:
# Let Knowledge Graph Agent extract node and relationship information
ans_element = kg_agent.run(element_example, parse_graph_elements=False)
print(ans_element)

 Nodes:

Node(id='CAMEL', type='Organization')
Node(id='Mistral AI', type='Organization')
Node(id='Mistral Large 2', type='Model')

Relationships:

Relationship(subj=Node(id='CAMEL', type='Organization'), obj=Node(id='Mistral AI', 
type='Organization'), type='developed')
Relationship(subj=Node(id='Mistral Large 2', type='Model'), obj=Node(id='Mistral AI', 
type='Organization'), type='runsWith')


In [86]:
chat_agent = ChatAgent(model=model)

# Example usage of the ChatAgent
prompt = "What is the capital of France?"
response = chat_agent.step(prompt)
print("Response:", response)

Response: msgs=[BaseMessage(role_name='assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content=' The capital of France is Paris.', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)] terminated=False info={'id': 'chatcmpl-227', 'usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 14, 'tool_calls': [], 'external_tool_call_requests': None}


C:\Users\This PC\AppData\Local\Temp\ipykernel_5288\843032406.py:5: RuntimeWarning: coroutine 'ChatAgent.astep' was never awaited
  response = chat_agent.step(prompt)


In [87]:
response.msgs[0].content

' The capital of France is Paris.'